<a href="https://colab.research.google.com/github/amanda-price/decoding-democracy/blob/master/scrapeTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is an example notebook. This would be an intro to what we are about to teach.

In [0]:
# Display the phrase "Hello World" to the monitor
print("Hello World")

Hello World


This way we can download the data, such as this: 

In [1]:
# Check to see if there is anything in the directory
!ls

# Make a new directory and download the file
!mkdir 2018
!ls

# Change directory
%cd '2018'

# Make sure we have changed path
!pwd

sample_data
2018  sample_data
/content/2018
/content/2018


In [2]:
# Change the url to the pdf you are interested in
!wget https://github.com/amanda-price/decoding-democracy/raw/master/2018/20180807_primary_precinct.pdf

#We will then display the list of files to make sure it downloaded
!ls

--2019-03-12 23:27:07--  https://github.com/amanda-price/decoding-democracy/raw/master/2018/20180807_primary_precinct.pdf
Resolving github.com (github.com)... 192.30.253.112, 192.30.253.113
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/amanda-price/decoding-democracy/master/2018/20180807_primary_precinct.pdf [following]
--2019-03-12 23:27:08--  https://raw.githubusercontent.com/amanda-price/decoding-democracy/master/2018/20180807_primary_precinct.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6325734 (6.0M) [application/octet-stream]
Saving to: ‘20180807_primary_precinct.pdf’

20180807_primary_pr 100%[===================>]   6.03M  --.-KB/s

In [15]:
# Install and load PDF scraper packages
!pip install PyPDF2
!pip install pdfminer.six
from PyPDF2 import PdfFileWriter, PdfFileReader

input1 = PdfFileReader(open("20180807_primary_precinct.pdf", "rb"))

# print how many pages input1 has:
print ("document1.pdf has {} pages.".format(input1.getNumPages()))


document1.pdf has 888 pages.


In [0]:
def scrapeOnePage(pageNum):
  
  pageObj = input1.getPage(pageNum)
  pdfWriter = PdfFileWriter()
  pdfWriter.addPage(pageObj)
  pdfOutputFile = open('onepage.pdf', 'wb')
  pdfWriter.write(pdfOutputFile)
  pdfOutputFile.close()

  !pdf2txt.py onepage.pdf > onepage.txt

  flag = False
  officeFlag = False
  strFlag = False
  colFlag = False
  numArray = []
  strArray = []
  colArray = []
  
  with open("onepage.txt") as file:
    for line in file:
      row = line.strip()
      #print("new row: {}".format(row))
      
      if row.startswith("Reg"):
        colFlag = True
      
      if row.isdigit():
        numFlag = True
      
      if flag:
        if row.startswith("W") or row.startswith("Polling") or row.startswith("ABSENTEE") or row.startswith("PROVISIONAL") or row.startswith("HOUSE") or row.startswith("Total"):
          if not row == "Total Votes":
            flag = False
            colFlag = False
            strFlag = True
          #print(colFlag)
          else:
            if not row == "":
              if not row.startswith("CITYWIDE") :
                #print(row)
                numArray.append(row)
      
      if officeFlag and not row == "":
        office = row
        officeFlag = False
        flag = True
        
      if colFlag:
        if not row == "":
          #print(row)
          colArray.append(row)
      if row.endswith("%"):
        splitting = row.split(" ")
        print("This is the split: {}".format(splitting))
      if row.startswith("Page:"):
        officeFlag = True
      
      
      if row.startswith("W"):
        
        print("Ward: {} Precinct: {} Office: {} Party: {}".format(row[2:4],row[7:9],office[:-3],office[-3:]))
        strFlag = True
      else:
        strArray.append(row)
        
      
  print("These are the columns for page {}: {}".format(pageNum,colArray))        
  #rint(numArray, strArray)        
  return numArray, strArray

    
  

In [9]:
# Try using PyPDF2's extractText() function

pageObj = input1.getPage(1)
thisPage = pageObj.extractText()
print(thisPage)

Statement of Votes CastPrimary ElectionSt. Louis, MissouriAugust 7, 2018SOVC For CITYWIDE, All Counters, All RacesFINAL OFFICIAL RESULTSDate:08/17/18Time:09:04:36Page:2 of 888        PROVISIONAL        HOUSE VOTES        Total    W 02 P 04        Polling        ABSENTEE        PROVISIONAL        HOUSE VOTES        Total    W 02 P 05        Polling        ABSENTEE        PROVISIONAL        HOUSE VOTES        Total    W 02 P 06        Polling        ABSENTEE        PROVISIONAL        HOUSE VOTES        Total    W 02 P 07        Polling        ABSENTEE        PROVISIONAL        HOUSE VOTES        Total    W 02 P 08        Polling        ABSENTEE        PROVISIONAL        HOUSE VOTES        Total    W 03 P 01        Polling        ABSENTEE        PROVISIONAL        HOUSE VOTES        Total    W 03 P 02        Polling        ABSENTEE        PROVISIONAL        HOUSE VOTES        Total    W 03 P 03        Polling        ABSENTEE        PROVISIONAL        HOUSE VOTES        Total    W 03 P 04 

In [116]:

officeList = []
for i in range(30,35):
  numArray, strArray = scrapeOnePage(i)
  if office in officeList:
    pass
  else:
    officeList.append(office)
  #print(numArray, strArray, wardArray)
  pos = position(strArray[0])
  length_num = 0

  #for l in numArray:
   # if not l.startswith("W"):
    #  length_num = length_num+1
  #length_num = int(length_num/3)
  current_pos = 0
  current_ward = 0
  
  
  totals = numArray[:length_num]
  #castVotes = numArray[length_num:2*length_num-2]
  #castVotes.append('280')
  #castVotes.append('13')
  #Doesnt seem to be getting 280 and 13 from the cards cast column (last two). This would be correct if it was there
  
  #for i in range(len(totals)):
    #print(strArray[current_pos],totals[current_pos],castVotes[current_pos])
    #current_pos = current_pos+1

    #if strArray[current_pos] == "Total":
      #current_ward = current_ward+1
    
#print(officeList)


    

Ward: 08 Precinct: 02 Office: US SENATOR  Party: REP
Ward: 08 Precinct: 03 Office: US SENATOR  Party: REP
Ward: 08 Precinct: 04 Office: US SENATOR  Party: REP
Ward: 08 Precinct: 05 Office: US SENATOR  Party: REP
Ward: 08 Precinct: 06 Office: US SENATOR  Party: REP
Ward: 08 Precinct: 07 Office: US SENATOR  Party: REP
Ward: 08 Precinct: 08 Office: US SENATOR  Party: REP
Ward: 08 Precinct: 09 Office: US SENATOR  Party: REP
Ward: 09 Precinct: 01 Office: US SENATOR  Party: REP
Ward: 09 Precinct: 02 Office: US SENATOR  Party: REP
This is the split: ['4', '12.90%']
This is the split: ['4', '14.29%']
This is the split: ['0.00%']
This is the split: ['4', '13.33%']
This is the split: ['0.00%']
This is the split: ['0.00%']
This is the split: ['0.00%']
This is the split: ['4', '12.12%']
This is the split: ['0.00%']
This is the split: ['4', '11.76%']
This is the split: ['7.69%']
This is the split: ['7.69%']
This is the split: ['8.33%']
This is the split: ['0.00%']
This is the split: ['6.67%']
This 